In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_read_xml as pdx
import xml.etree.ElementTree as ET
from lxml import objectify 
import re



In [ ]:
#Some offending characters had to be removed manually
df = pdx.read_xml('data/Sabanews_utf_8.xml', ['SabanewsData', 'Sabanews'],
                 root_is_rows=False)

In [33]:
df.head(5)

,ID,URL,Headline,Dateline,Text
0,SBN_ARB_0000001,http://www.sabanews.net/ar/news200024.htm,الكونجرس الأمريكي يطالب المجتمع الدولي دعم الي...,08/ديسمبر/2009,08/ديسمبر/2009] واشنطن ـ سبأنت: طالب الكونجرس ...
1,SBN_ARB_0000002,http://www.sabanews.net/ar/news200066.htm,مناقشة إستراتيجية التواصل الخاصة بالأجندة الوط...,08/ديسمبر/2009,08/ديسمبر/2009] صنعاء ـ سبأنت: جرى اليوم بوزار...
2,SBN_ARB_0000003,http://www.sabanews.net/ar/news200073.htm,نائب وزير التخطيط يلتقي بعثة الوكالة اليابانية...,08/ديسمبر/2009,08/ديسمبر/2009] صنعاء ـ سبأنت: أشاد نائب وزير ...
3,SBN_ARB_0000004,http://www.sabanews.net/ar/news200090.htm,الناطق الرسمي: الرؤية الحكيمة للدولة في التعام...,08/ديسمبر/2009,08/ديسمبر/2009] صنعاء – سبأنت : أكد الناطق الر...
4,SBN_ARB_0000005,http://www.sabanews.net/ar/news200092.htm,نوكيا تطرح خدمة الخرائط والملاحة في الأسواق,08/ديسمبر/2009,08/ديسمبر/2009] القاهرة ـ سبأنت: عبداللطيف الك...


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92149 entries, 0 to 92148
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        92149 non-null  object
 1   URL       92149 non-null  object
 2   Headline  92149 non-null  object
 3   Dateline  92149 non-null  object
 4   Text      92149 non-null  object
dtypes: object(5)
memory usage: 3.5+ MB


In [35]:
def clean_text(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']  
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
        
    text = text.strip()
    
    return text

In [39]:
df['clean_text'] = df['Text'].apply(lambda x: clean_text(x))

In [66]:
#removes date from main body of text
df['new_clean_text'] = df['clean_text'].apply(lambda x: x[x.find(']')+1:])